In [1]:
import csv
import nltk
from nltk.collocations import *
from nltk.metrics.spearman import *
trigram_measures = nltk.collocations.TrigramAssocMeasures()
stopwords = nltk.corpus.stopwords.words('russian')

In [2]:
arr = []
with open ('court-V-N.csv', newline='') as csvfile:
    reader1 = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader1:
        print(''.join(row).split(','))
        arr.append(''.join(row).split(','))
    

['СУД', 'АРЕСТОВАТЬ', 'ПОДПОЛКОВНИК', 'ЗАМНАЧАЛЬНИК', 'ОТДЕЛ']
['СУД', 'ВЕСТИ', 'ПЕРЕГОВОРЫ', 'ПОСТАВКА', 'БУДУЩИЙ']
['СУД', 'ВЗЯТЬ', 'РЕАЛИЗАЦИЯ', 'ЗАКОН', 'ОБЯЗАТЕЛЬНЫЙ']
['СУД', 'ВЫКУПИТЬ', 'ГОСУДАРСТВО', 'АКЦИЯ', 'РАНЕЕ']
['СУД', 'ГОВОРИТЬ', 'ПРЕСС-СЕКРЕТАРЬ', 'ЕВГЕНИЙ', 'ВРЕМЕНИТЬ']
['СУД', 'ЕСТЬ', 'ЗВАНИЕ', 'ЗАСЛУЖЕННЫЙ', 'МАШИНОСТРОИТЕЛЬ']
['СУД', 'ЗАЯВИТЬ', 'ЭНЕРГИЯ', 'ПРОВОДИТЬ', 'НЕСКОЛЬКО']
['СУД', 'ИЗМЕНИТЬ', 'СИТУАЦИЯ', 'ПЕРИОД', 'НЕМЕЦКИЙ']
['СУД', 'НАПОМНИТЬ', 'ТРЕБОВАНИЕ', 'ОБЪЯСНЯТЬ', 'СОГЛАШЕНИЕ']
['СУД', 'ОБВИНИТЬ', 'РАБОТОДАТЕЛЬ', 'ПРИМЕНЕНИЕ', 'СЕРЫЙ']
['СУД', 'ОБВИНЯТЬ', 'ДЕЗИНФОРМАЦИЯ', 'СУД', 'АДВОКАТ']
['СУД', 'ОБРАЩАТЬ', 'ВНИМАНИЕ', 'ВЫСТУПЛЕНИЕ', 'ИМЕТЬ']
['СУД', 'ОБСУДИТЬ', 'ХОД', 'ДУМСКИЙ', 'КАМПАНИЯ']
['СУД', 'ОБЪЕДИНИТЬ', 'КОМПАНИЯ', 'РАСТИТЬ', 'РАСТИ']
['СУД', 'ОЖИДАТЬ', 'СВЯЗЬ', 'ИЗМЕНЕНИЕ', 'ВКАЛЫВАТЬ']
['СУД', 'ОПРЕДЕЛИТЬСЯ', 'ВРАГ', 'НАРОД', 'КОЛЛАЖ']
['СУД', 'ОСУДИТЬ', 'ПОЛКОВНИК', 'ОТВЕТ', 'РУКОВОДСТВО']
['СУД', 'ПИСАТЬ', 'ПОЧТА', 'ИСКАТЬ', 'ИНФОРМ

In [3]:
words = []
for i in arr:
    for j in i:
        words.append(j)
        

Золотой стандарт я составила на основе частотности триграмм в НКРЯ (при условии, что эти триграммы есть в исходном файле) и интуиции.

In [4]:
gold = [('СУД', 'РАССМАТРИВАТЬ', 'ДЕЛО'),('СУД', 'ВЫНЕСТИ', 'РЕШЕНИЕ'), ('СУД', 'ПРИНЯТЬ', 'РЕШЕНИЕ'),
        ('СУД', 'УДОВЛЕТВОРИТЬ', 'ИСК'), ('СУД', 'ВЫНЕСТИ', 'ПРИГОВОР'), ('СУД', 'ПРИЗНАТЬ', 'ВИНОВНАЯ'), 
        ('СУД', 'ПРИЗНАТЬ', 'ПРАВОТА'), ('СУД', 'ОТКЛОНИТЬ', 'ИСК'), 
        ('СУД', 'ПРИЗНАТЬ', 'РЕШЕНИЕ'), ('СУД', 'РАССМОТРЕТЬ', 'ХОДАТАЙСТВО')
         ]

In [5]:
tr_finder = TrigramCollocationFinder.from_words(words)
tr_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stopwords)
tr_finder.apply_freq_filter(3)


Log-likelihood (вот это штукой 'i[0][2] !='СУД'' исключаем коллокации вида "суд есть суд", 
                которые не описывают действия суда)

In [6]:
sud_ll = []
tr_finder.nbest(trigram_measures.likelihood_ratio, 10)
scored_trigrams = tr_finder.score_ngrams(trigram_measures.likelihood_ratio)
scored_trigrams = [i for i in scored_trigrams if i[0][0] == 'СУД' and i[0][2] !='СУД' and i[0] in gold]
for i in scored_trigrams:
    sud_ll.append(i[0])

In [7]:
print('%0.2f' % spearman_correlation(ranks_from_sequence(sud_ll), ranks_from_sequence(gold)))

-0.21


Mi-like

In [8]:
tr_finder.nbest(trigram_measures.mi_like, 10)
sud_mi = []
scored_trigrams = tr_finder.score_ngrams(trigram_measures.mi_like)
scored_trigrams = [i for i in scored_trigrams if i[0][0] == 'СУД' and i[0][2] !='СУД' and i[0] in sud_ll]
for i in scored_trigrams[:10]:    
    sud_mi.append(i[0])

In [9]:
print('%0.2f' % spearman_correlation(ranks_from_sequence(sud_mi), ranks_from_sequence(gold)))

0.22


Frequency

In [10]:
tr_finder.nbest(trigram_measures.raw_freq, 10)
sud_raw = []
scored_trigrams = tr_finder.score_ngrams(trigram_measures.raw_freq)
scored_trigrams = [i for i in scored_trigrams if i[0][0] == 'СУД' and i[0][2] !='СУД' and i[0] in sud_ll]
for i in scored_trigrams[:10]:
    sud_raw.append(i[0])

In [11]:
print('%0.2f' % spearman_correlation(ranks_from_sequence(sud_raw), ranks_from_sequence(gold)))

0.22


Pmi

In [12]:
tr_finder.nbest(trigram_measures.pmi, 10)
sud_pmi = []
scored_trigrams = tr_finder.score_ngrams(trigram_measures.pmi)
scored_trigrams = [i for i in scored_trigrams if i[0][0] == 'СУД' and i[0][2] !='СУД' and i[0] in sud_ll]
for i in scored_trigrams[:10]:
    sud_pmi.append(i[0])

In [13]:
print('%0.2f' % spearman_correlation(ranks_from_sequence(sud_pmi), ranks_from_sequence(gold)))

-0.30


Из четырёх использованных метрик более-менее хорошие результаты (и удивительно схожие) показали частота и mi-like. 
Pmi и LL наоборот в минусе.


Насчёт того, почему коэффициенты pmi и mi_like противоположны, понятно
(эти функции противоположны), mi -- mutual information больше соответствует интуиции.
С частотностью тоже всё ясно -- GS отбирался по частоте встречаемости в НКРЯ, поэтому коэффициент положительный.